In [1]:
import gurobipy as gp
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from markowitz import Markowitz
from markowitz import par

In [2]:
def limpa(s):
    return s.replace("/","").replace("-","").replace(".","").replace("$","").replace("R","")

def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))

In [3]:
df = pd.read_excel('seriehistorica10anos.xlsx')

In [4]:
df.fillna(method='ffill',inplace=True)
df.rename(columns={df.columns[0]: 'datas'},
          inplace=True, errors='raise')

In [5]:
datas = list(df['datas'][:])
aux = [i[0:4] for i in datas]
anos = pd.Series(aux).unique().tolist()

limites_anos = [0]

a = 0

for c in anos:
    a += aux.count(c)
    limites_anos.append(a)

print(anos)

['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


In [6]:
DF_anos = dict()

for k in range(len(limites_anos)-1):
    DF_anos[anos[k]] = df[limites_anos[k]:limites_anos[k+1]].reset_index().drop(['index'],axis=1)

In [7]:
DF_meses = dict()
Meses = dict()

for j  in range(len(limites_anos)-1):
    year = anos[j]
    datas = list(DF_anos[year]['datas'][:])
    aux = [i[5:7] for i in datas]
    Meses[year] = pd.Series(aux).unique().tolist()

    limites_meses = [0]

    a = 0

    for c in Meses[year]:
        a += aux.count(c)
        limites_meses.append(a)
        
    for k in range(len(limites_meses)-1):
        DF_meses[year,Meses[year][k]] = DF_anos[year][limites_meses[k]:limites_meses[k+1]].reset_index().drop(['index'],axis=1)

In [18]:
minRetorno = 0.2
C = 100_000
P_categorias = [0.25, 0.05, 0.4, 0.3]
K_min = 3
K_max = 10
P_min = 0.05
P_max = 0.3
l = 1
log = 0

In [19]:
sum(P_categorias)

1.0

In [20]:
ano_inicial = 2011
ano_final = 2014
mes_inicial = 9
mes_final = 8
intervalo_mensal = 12

In [21]:
anos_escolhidos = anos[anos.index(str(ano_inicial)):(ano_final-ano_inicial+1+anos.index(str(ano_inicial)))]
print(anos_escolhidos)
seq = []
for k  in anos_escolhidos:
    if k == str(ano_final):
        seq += [(k,q) for q  in Meses[k][0:mes_final]]
    else:
        seq += [(k,q) for q  in Meses[k]]
print(len(seq)/intervalo_mensal)
print(seq)

['2011', '2012', '2013', '2014']
3.0
[('2011', '09'), ('2011', '10'), ('2011', '11'), ('2011', '12'), ('2012', '01'), ('2012', '02'), ('2012', '03'), ('2012', '04'), ('2012', '05'), ('2012', '06'), ('2012', '07'), ('2012', '08'), ('2012', '09'), ('2012', '10'), ('2012', '11'), ('2012', '12'), ('2013', '01'), ('2013', '02'), ('2013', '03'), ('2013', '04'), ('2013', '05'), ('2013', '06'), ('2013', '07'), ('2013', '08'), ('2013', '09'), ('2013', '10'), ('2013', '11'), ('2013', '12'), ('2014', '01'), ('2014', '02'), ('2014', '03'), ('2014', '04'), ('2014', '05'), ('2014', '06'), ('2014', '07'), ('2014', '08')]


In [22]:
Hist = dict()
for i in range(int(len(seq)/intervalo_mensal)):
    DF = pd.DataFrame()
    for j in range(intervalo_mensal):
        print(seq[intervalo_mensal*i+j])
        DF = pd.concat([DF, DF_meses[seq[intervalo_mensal*i+j]]], axis=0).reset_index().drop(['index', 'datas'],axis=1)
        
    pf = pd.read_excel('perfilfundos.xlsx')
    pf.drop(['NOME','DATA_INICIO','CARENCIA','FLAG_QUALIFICADO'],axis=1,inplace=True)
    pf['CNPJ'] = pf['CNPJ'].apply(limpa)
    pf['APLICACAO_MINIMA'] = pd.to_numeric(pf['APLICACAO_MINIMA'].apply(limpa),errors='coerce')
    pf.fillna(0,inplace=True)
    pf.set_index('CNPJ',inplace=True)
    pf_ordered = pf.sort_values(by ='CATEGORIA')
    pf_ordered.drop(index=(list(set(pf.index)-set(DF.columns))),inplace=True)

    categorias = list(pf_ordered['CATEGORIA'].unique())

    limites = [0]

    a = 0

    for c in categorias:
        a += len(pf_ordered[pf_ordered['CATEGORIA'] == c])
        limites.append(a)

    cnpj = list(pf_ordered.index)
    DF = DF[cnpj]

    df_retorno = DF.pct_change().dropna()
    sigma = (df_retorno.cov()*252).to_numpy()
    media = (df_retorno+1).apply(geo_mean)**252-1

    minFundos = np.array(pf_ordered['APLICACAO_MINIMA'])

    n = len(minFundos)

    parametros = par(C, minRetorno, K_min, K_max, P_min, P_max, P_categorias, limites, n, log)
    parametros.Exibir_par()

    import time

    tempo_init = time.time()
    Carteira = Markowitz(minFundos, sigma, media, cnpj, parametros);
    print('Tempo total:', time.time() - tempo_init, 's')

    Carteira.Exibir(C)

    Hist[seq[intervalo_mensal*i]] = Carteira
    print('\n')
    print('###############################################')
    print('\n')

('2011', '09')
('2011', '10')
('2011', '11')
('2011', '12')
('2012', '01')
('2012', '02')
('2012', '03')
('2012', '04')
('2012', '05')
('2012', '06')
('2012', '07')
('2012', '08')
P_min = 5.0%
P_max = 30.0%
K_min = 4
K_max = 10
P_categorias = [0.25, 0.05, 0.4, 0.3]
Tempo total: 0.32942962646484375 s
Retorno esperado --> 20.0%
Risco estimado --> 2.33%
Fundo 5 (12287682000144) --> 8.75%; R$ 8753.82 --> R$ 5000.0
Fundo 25 (10237480000162) --> 7.58%; R$ 7576.3 --> R$ 1000.0
Fundo 52 (12004203000135) --> 8.67%; R$ 8669.87 --> R$ 5000.0
Fundo 60 (01430938000104) --> 5.0%; R$ 5000.0 --> R$ 5000.0
Fundo 68 (13001211000190) --> 22.82%; R$ 22824.39 --> R$ 5000.0
Fundo 85 (06041290000106) --> 11.24%; R$ 11235.2 --> R$ 1000.0
Fundo 125 (12120451000141) --> 5.94%; R$ 5940.41 --> R$ 1000.0
Fundo 128 (09814233000100) --> 7.25%; R$ 7246.79 --> R$ 3000.0
Fundo 149 (10783480000168) --> 10.77%; R$ 10770.13 --> R$ 100.0
Fundo 150 (13554383000191) --> 11.98%; R$ 11983.08 --> R$ 5000.0


###################